<a href="https://colab.research.google.com/github/Geuens/curso_ai/blob/main/Copy_of_embeddings_stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-secret-manager google-auth google-auth-oauthlib openai kaggle nltk

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
import os

# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
if not os.path.exists('/.kaggle'):
    !mkdir /.kaggle
    !mv kaggle.json /.kaggle
    !mv /.kaggle /root/
    !chmod 600 ~/.kaggle/kaggle.json

if not os.path.exists('60k-stack-overflow-questions-with-quality-rate.zip'):
  !kaggle datasets download imoore/60k-stack-overflow-questions-with-quality-rate
  !unzip *.zip

df = pd.read_csv("/content/train.csv")
df = df[:200]
print(df.shape)
df.head()

(200, 6)


,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [ ]:
import openai
from openai import OpenAI
from google.cloud import secretmanager
from google.colab import auth
auth.authenticate_user()
# Creamos un Cliente de SecretManager:
client = secretmanager.SecretManagerServiceClient()
secret_name = "openai-token" # => Cambia por el nombre de tu secreto (en google cloud secret manager)
project_id = '196177837977' # => Cambia por el id de tu proyecto Google Cloud

# Construye un F-string con los datos:
resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"

# REcupera tu secreto :
response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')

# Tu clave secreta queda bajo secret_string

# Configurar el motor de OpenAI
engine = "gpt-3.5-turbo"
client = OpenAI(api_key=secret_string)

def get_completion(prompt):
  completion = client.chat.completions.create(
  model=engine,
  messages=[
      {"role": "system", "content": "Eres un asistente, que realizas resúmenes concisos y proporcionas la ideas principales de un texto."},
      {"role": "user", "content": f"{prompt}"}
    ]
  )
  return completion

def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    # Obtener el embedding del primer elemento de la respuesta
    embedding = response.data[0].embedding
    return embedding

In [ ]:
# Obtenemos los embeddings para los títulos
if not os.path.exists('stack_overflow_embeddings.pkl'):
    df['embedding'] = df['Title'].apply(get_embedding)
# Guardamos a un archivo para no tener que volver a pedir los embedding a openai
df.to_pickle("stack_overflow_embeddings.pkl")

## Usamos esta vez Word2Vec

In [ ]:

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocesar(texto):
    palabras = word_tokenize(texto.lower())
    return [palabra for palabra in palabras if palabra.isalpha() and palabra not in stop_words]

df['Title_preprocesado'] = df['Title'].apply(preprocesar)

def limpiar_etiquetas(tags_str):
    # Eliminar los símbolos '<' y '>' y dividir en palabras individuales
    return tags_str.replace('>', ' ').replace('<', '').split()

# Aplica la función a tu columna de etiquetas
df['Tags_clean'] = df['Tags'].apply(limpiar_etiquetas)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from gensim.models import Word2Vec

# Entrenar el modelo Word2Vec
modelo_etiquetas = Word2Vec(df['Tags_clean'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd

# Cargar los embeddings guardados
df = pd.read_pickle("stack_overflow_embeddings.pkl")

def promedio_embeddings_etiquetas(etiquetas, modelo):
    embeddings = [modelo.wv[etiqueta] for etiqueta in etiquetas if etiqueta in modelo.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(modelo.vector_size)

df['Tags_embedding'] = df['Tags_clean'].apply(lambda etiquetas: promedio_embeddings_etiquetas(etiquetas, modelo_etiquetas))

# Concatenar los embeddings de las preguntas y las etiquetas
df['combined_embedding'] = df.apply(lambda fila: np.concatenate([fila['embedding'], fila['Tags_embedding']]), axis=1)

def recomendar_preguntas_similares(embedding_nueva_pregunta, embeddings_existentes, top_n=5):
    # Calcula la similitud coseno entre la nueva pregunta y todas las existentes
    similitudes = [1 - cosine(embedding_nueva_pregunta, emb) for emb in embeddings_existentes]

    # Ordena las preguntas por similitud
    indices_ordenados = np.argsort(similitudes)[::-1]

    # Retorna los top_n índices más similares
    return indices_ordenados[:top_n]

# Ejemplo de uso
embedding_nueva_pregunta = get_embedding("What is a tuple in python?")  # Obtén el embedding de la API de OpenAI
tags_nueva_pregunta = "<python> <programming>"  # Etiquetas de la nueva pregunta
tags_nueva_pregunta_embedding = promedio_embeddings_etiquetas(limpiar_etiquetas(tags_nueva_pregunta), modelo_etiquetas)

# Combina el embedding de la pregunta con el de las etiquetas
combined_embedding_nueva_pregunta = np.concatenate([embedding_nueva_pregunta, tags_nueva_pregunta_embedding])

# Encuentra preguntas relacionadas
indices_recomendados = recomendar_preguntas_similares(combined_embedding_nueva_pregunta, df['combined_embedding'].tolist())
preguntas_recomendadas = df.iloc[indices_recomendados]

print(preguntas_recomendadas['Title'])

46                  Has set been deprecated in python 2?
127    Why do we use print statements while defining ...
93       Python for loop - to iterate or not to iterate?
165    Print type of variable in python3.5 on django 1.8
193          python 2.7 : find nested keys in dictionary
Name: Title, dtype: object
